In [1]:
import numpy as np

from option import Option

In [2]:
class FiniteDifferencesPricer:
    """
    Finite Differences pricer based on the Black Scholes Model.
    """
    
    ############################### INIT ####################################
    def __init__(self, verbose=False):
        
        self.verbose = verbose
        
        
    def setMarketParameters(self, r, sigma, T, x_min, x_max):
        self.r = r
        self.sigma = sigma
        self.T = T
        self.x_min = x_min
        self.x_max = x_max
    
    
    def setMesh(self, I, N):
        self.I = I
        self.N = N
        
        self.h = (self.x_max - self.x_min)/self.I
        self.dt = self.T/self.N
        
        self.x = np.linspace(self.x_min, self.x_max, self.I+1)
        self.t = np.linspace(0, self.T, self.N+1)
    #########################################################################
    
    ####################### TOOLS FOR SIMULATIONS ###########################
    def compute_A(self):
        self.alpha = self.sigma**2/(2 * self.h**2) * self.x**2
        self.beta = self.r/(2 * self.h) * self.x
        
        sub_diag = (- self.alpha + self.beta)[2:self.I+1]
        main_diag = 2 * self.alpha[1:self.I+1] + self.r
        sup_diag = (- self.alpha - self.beta)[1:self.I]
        self.A = np.diag(sub_diag, k=-1) + np.diag(main_diag) + np.diag(sup_diag, k=1)
    
    def compute_q(self, t, option):
        q = np.zeros(self.I)
        q[0] = (- self.alpha[1] + self.beta[1]) * option.left_boundary(t, self.x_min)
        q[-1] = (- self.alpha[self.I] - self.beta[self.I]) * option.right_boundary(t, self.x_max)
        
        return q

    def interpolate(self, x_val, U):
        i = np.searchsorted(self.x, x_val, side='right') - 1
        return (self.x[i+1] - x_val)/self.h * U[i] + (x_val - self.x[i])/self.h * U[i+1]
    #########################################################################
    
    
    ############################## FINITE SCHEMES ###########################
    def computeEulerScheme(self, U_0, option):
        
        self.compute_A()
        U = np.zeros((self.N+1, self.I))
        U[0] = U_0
        for n in range(self.N):
            U[n+1] = np.linalg.solve(self.dt * self.A + np.eye(self.I), U[n] - self.dt * self.compute_q(self.t[n], option))

        return U[self.N]
    
    def computeBDFScheme(self, U_0, option):
        
        self.compute_A()
        U_1 = np.maximum(U_0, np.linalg.solve(self.dt * self.A + np.eye(self.I), U_0 - self.dt * self.compute_q(self.dt, option)))
        
        U = np.zeros((self.N+1, self.I))
        U[0] = U_0
        U[1] = U_1
        for n in range(1, self.N):
            
            # We want to solve M U^(n+1) = N U^n + P U^(n-1) + Q
            M = 3 * np.eye(self.I) + 2 * self.dt * self.A
            N, P = 4, -1
            Q = -2 * self.dt * self.compute_q(self.t[n] + self.dt, option)
            
            U[n+1] = np.maximum(U_0, np.linalg.solve(M, N*U[n] + P * U[n-1] + Q))
        
        return U[self.N]
    #########################################################################
    
    def price(self, option, x_val):
        
        g = option(self.x[:-1])
        
        if option.kind == 'European':
            U = self.computeEulerScheme(g, option)
        elif option.kind == 'American':
            U = self.computeBDFScheme(g, option)
            
        return self.interpolate(x_val, U)

## American Put Option

In [3]:
def put_payoff(x, strike=40):
    return np.maximum(strike - x, 0)

def put_left_condition(t, x_min, strike=40):
    return np.maximum(strike - x_min, 0)

def put_right_condition(t, x_max):
    return 0

In [4]:
pricer = FiniteDifferencesPricer()
pricer.setMarketParameters(r=0.06, sigma=0.2, T=1, x_min=1, x_max=100)
pricer.setMesh(I=128, N=1280)

In [5]:
put_option = Option(payoff = put_payoff,
                    maturity = 1.,
                    kind = 'European',
                    exercising_times = np.array([1.]),
                    left_boundary = put_left_condition,
                    right_boundary = put_right_condition)
pricer.price(put_option, 36)

3.8791155221963165

In [6]:
put_option = Option(payoff = put_payoff,
                    maturity = 1.,
                    kind = 'American',
                    exercising_times = np.array([1.]),
                    left_boundary = put_left_condition,
                    right_boundary = put_right_condition)
pricer.price(put_option, 36)

4.520821880633168

## American square put options

In [7]:
def square_payoff(x, strike=40):
    return np.maximum(strike**2 - x**2, 0)

def square_left_condition(t, x_min, strike=40):
    return np.maximum(strike**2 - x_min**2, 0)

def square_right_condition(t, x_max):
    return 0

In [8]:
pricer = FiniteDifferencesPricer()
pricer.setMarketParameters(r=0.06, sigma=0.2, T=1, x_min=1, x_max=100)
pricer.setMesh(I=128, N=1280)

In [9]:
square_option = Option(payoff = square_payoff,
                       maturity = 1.,
                       kind = 'European',
                       exercising_times = np.array([1.]),
                       left_boundary = square_left_condition,
                       right_boundary = square_right_condition)
pricer.price(square_option, 36)

273.94301307236094

In [10]:
square_option = Option(payoff = square_payoff,
                       maturity = 1.,
                       kind = 'American',
                       exercising_times = np.array([1.]),
                       left_boundary = square_left_condition,
                       right_boundary = square_right_condition)
pricer.price(square_option, 36)

333.27792807294253